# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy.crs as ccrs
import geoviews as gv
import pyproj

# Import API key
from api_keys import geoapify_key

ModuleNotFoundError: No module named 'pyproj._network'

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aswan,24.0934,32.9070,98.04,14,11,4.92,EG,1719559160
1,1,labytnangi,66.6572,66.4183,53.71,52,100,12.10,RU,1719559161
2,2,sabha,27.0377,14.4283,88.75,16,0,13.31,LY,1719559163
3,3,leo,11.1000,-2.1000,78.64,74,100,8.86,BF,1719559164
4,4,labuan,5.2767,115.2417,88.16,70,20,3.44,MY,1719559165


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,  
    size = "Humidity",
    color = "City"
)

# Display the map plot
map_plot_1

ImportError: Geographic projection support requires GeoViews and cartopy.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] < 80) 
                                    & (city_data_df["Max Temp"] > 70) 
                                    & (city_data_df["Wind Speed"] < 8)
                                    & (city_data_df["Cloudiness"] < 30)
                                    & (city_data_df["Humidity"] < 60),:].dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
92,92,calatayud,41.3535,-1.6432,74.23,52,2,5.59,ES,1719559269
222,222,kittila,67.6647,24.8936,70.61,40,0,6.91,FI,1719559228
270,270,ankazoabo,-22.2833,44.5167,72.82,29,0,6.29,MG,1719559485
312,312,laguna,38.4210,-121.4238,71.15,56,0,5.75,US,1719559513
346,346,vieste,41.8825,16.1750,79.29,57,0,6.58,IT,1719559583
397,397,newman,37.3138,-121.0208,70.03,58,0,3.00,US,1719559641


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
92,calatayud,ES,41.3535,-1.6432,52,
222,kittila,FI,67.6647,24.8936,40,
270,ankazoabo,MG,-22.2833,44.5167,29,
312,laguna,US,38.4210,-121.4238,56,
346,vieste,IT,41.8825,16.1750,57,
397,newman,US,37.3138,-121.0208,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
            "limit": 20,
            "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
calatayud - nearest hotel: Arco San Miguel
kittila - nearest hotel: Hotelli Kittilä
ankazoabo - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
vieste - nearest hotel: Palace Hotel Vieste
newman - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
92,calatayud,ES,41.3535,-1.6432,52,Arco San Miguel
222,kittila,FI,67.6647,24.8936,40,Hotelli Kittilä
270,ankazoabo,MG,-22.2833,44.5167,29,No hotel found
312,laguna,US,38.4210,-121.4238,56,Holiday Inn Express & Suites
346,vieste,IT,41.8825,16.1750,57,Palace Hotel Vieste
397,newman,US,37.3138,-121.0208,58,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,  
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
map_plot_2

ImportError: Geographic projection support requires GeoViews and cartopy.